**Introduction to Bioinformatics**<br>
A masters course by Blaž Zupan and Tomaž Curk<br>
University of Ljubljana (C) 2016-2018

Disclaimer: this work is a first draft of our working notes. The images were obtained from various web sites, but mainly from the wikipedia entries with explanations of different biological entities. Material is intended for our bioinformatics class and is not meant for distribution.

## Lecture Notes Part 8

# Hidden Markov Models, State Probabilities and Posterior Decoding

Hidden Markov Models provide us with means to model processes with unobserved (hidden) states and observed sequences of symbols that are emitted from the model. In the previous lesson we have learned how to define the model, use it to generate both sequences (the path and the sequence of emitted symbols) and compute their joint probabilities assuming that we know what the path was (which we, in general, do not). Then we have developed a method to find the most probable path, that is

$$\pi^*={\rm argmax}_\pi^ P(x, \pi)$$

For this, we introduced Viterbi algorithm that is for each element of the sequence and for each possible state computes the probability of the most probable path until this state, $v_k(i-1)$. For example, for our loaded dice problem, and a sequence of observed states

```
-o--ooo-o--o-o---ooooooo-ooooo-oooooooooooo-oo-o--oooooooooooooooo-oooooooo-ooooooooo-o-oo--o-ooooo-
```

the decoded path by Viterbi algorithm is

```
FFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFLLLLLFF
```

For this particular case, because we have simulated the model to obtain the emissions, we actually know what the path was:

```
FFFFFFFFFFFFFFFFFLLLLFFFFFFLLLLLLLFFLLLLLLLLLFFFFFLLLLLLLLLLLLLLLLFLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFLLF
```

So, not perfect, but the decoding did actually a fine job.

Viterbi decoding is fine, but also has problems. For one, it returns only one path, only one sequence of hidden states, although many similarly probable paths exist. The sequence $\pi^*$ is an informative, but does not give us the whole picture. In fact, we would be better off to know the probability of a state at certain time $i$. Here, we work towards this end and then present an algorithm for posterior decoding.

## Probability of a sequence $x$ by marginalization

On a surface, this is a simple question. Given the joint probability $P(x,\pi)$ we just have to compute the sum across all different paths:

$$P(x)=\sum_x P(x, \pi)$$

We can do this because, given the model, we know how to compute joint probability if both the path and the emissions are know:

$$P(x,\pi)=a_{0\pi_1}\prod_{i=1}^L e_{\pi_i}(x_i)a_{\pi_i \pi_{i+1}}$$

(this is not a new equation, we have already introduced it in the last lecture).

But, alas, we can not use this procedure since there are, for any reasonably-sized sequence, too many paths. So the above [marginalization](https://en.wikipedia.org/wiki/Marginal_distribution) does not work and we have to find something better.

## The forward algorithm

Say that we are given the probability of observed sequence up-to and including symbol $x_i$, where we have finished at the state $\pi_i=k$:

$$f_k(i)=P(x_1\ldots x_i,\pi=k)$$

then

$$f_l(i+1)=e_l(x_{i+1})\sum_k f_k(i) a_{kl}$$

At the time instance $i+1$, if we assume that we are in the state $l$, we come from time instance $i$ but we do not know what was the state. So we consider all possible states at time $i$ and their probabilities, multiply them with probability of transition from these states, and then sum these products out to get the probability that we transition to the state $l$. At this state, we emit symbol $x_{i+1}$, for which we know the probability of, so we have to account this in as well.

You can already see the procedure we have just defined is a recursive one, and that we can implement it through dynamic programming. Here is the formalization:

Initialization:
$f_0(0)=0$

Recursion ($i=1\ldots L$):
$$f_l(i)=e_l(x_i)\sum_k f_k(i-1)a_{kl}$$

Termination:
$$P(x)=\sum_k(f_k(L)a_{L0}$$

So, for any emitted sequence, we can compute the probability that this was generated from a particular hidden Markov model. The forward algorithm takes the sequence and the (fully specified) model, and computes the probability by which this model would generate the sequence. It elegantly, through dynamic programming, traverses through all possible hidden paths that exists, and uses Markov property to turn compute this through dynamic programming in linear time.

## What is the most probable state at observation $x_i$?

In fact, an even cooler question which we will try to answer is what is the probability that $x_i$ was emitted at state $\pi_i=k$. By this, we will also answer the question what is the probability of the most probable state at that time instance.

Before we nail this down, consider how more fancy is this when compared to Viterbi decoding $\pi^*$. This later is just a sequence, but know we will be able to know how do the state probabilities change. By plotting this probability, we nicely, say, see what are the most probable annotations across the genomes, and see how this probabilities change, in a continuous way, across the sequence.

We denote this probability with $P(\pi_i=k|x)$, and we call it a posterior probability, that is, the conditional probability that is assigned after considering relevant data, in our case after we consider the observations. Different would be prior probability, that is, the probability of the states that we would compute without observing any data, just by using the model. But this later is not what we are interested in. (This was just an attempt to explain the difference between *prior* and *posterior* probability).

So, we need to compute $P(\pi_i=k|x)$, the conditional probability of state at time $i$ being equal to $k$ if we have observed the sequence $x$. Perhaps we can start from joint probability $P(\pi_i=k,x)$, that is, the probability that the state at time $i$ is $k$ and the emitted sequence is $x$. Notice that if we would know the joint probability, we would be able to compute the conditional probability by:

$$ P(\pi_i=k|x)= P(\pi_i=k,x) / P(x) $$

Ah, but we need $P(x)$. No problem, we have it: we have just derived it using the forward algorithm. So let us concentrate on joint probability, and use some knowledge on probability laws, like $P(A|B)=P(AB)/P(B)$ to write it out:

$$P(x,\pi_i=k)=P(x_1 x_2\ldots x_i,\pi_i=k)\times P(x_{i+1}\ldots x_L|x_1\ldots x_i,\pi_i=k)\\
=P(x_1 x_2 \ldots x_i, \pi_i=k) \times P(x_i+1\ldots x_L|\pi_i=k)$$

The first term in this equation, P(x_1 x_2 \ldots x_i, \pi_i=k), is a probability that we have arrived at state $\pi_i=k$ and observed all the emitted symbols until this time, including $x_i$. Oh, but that is exactly the forward probability $f_k(i)$ that we have computed in the previous section. Great, this is done, then.

The remaining part, $P(x_i+1\ldots x_L|\pi_i=k)$, is something almost the opposite of forward. If the forward algorithm looks at the sequence until the point $i$, we need a similar algorithm that determines the probability for the rest of the sequence, given that we start with the state $k$ at $\pi_i$. Let us denote this probability with $b_k(i)$ and call it backward probability, and refer to the algorithm that will compute it as the backward algorithm. The algorithm is not too much different from the forward one:

Initialization:
$f_k(L)=a_{k0}$ for all $k$

Recursion ($i=L-1\ldots 1$):
$$b_k(i)=\sum_k a_{kl} e_l(x{i+1}) b_l{i+1}$$

Now for the wrap-up, since we have everything what we need:

$$P(\pi_i=k|x)={f_k(i)\times b_k(i)\over P(x)}$$

## Posterior Decoding

Remember: the Viterbi decoding finds the path with the highest joint probability with a given emission sequence. Now that we know how to compute the most probable state given the sequence, we can also reconstruct the path as one composed of most probable states:

$$\hat{\pi}_i={\rm argmax}_k P(\pi_i=k|x) \\
= {\rm argmax}_k f_k(i)\times b_k(i) $$

The problem with $\hat{\pi}$ is it may not be legitimate, given the hidden Markov model, as some transitions in the path may actually not be allowed (if some $a_{kl}=0$). Notice also that posterior decoding is different from Viterbi and will not produce the path with highest joint probability with the observed sequence.

## Parameter Estimation and Training

Actually, we can also call it machine learning of hidden Markov model. The premise here is that if we are given both sequence and the path, we can estimate the transition probabilities $a_{kl}$ and $e_k(b)$ from the counts $A_{kl}$ and $E_k(b)$. Here, $A_{kl}$ is the number of times we observe a pair $kl$ in the path and $E_k(b)$ is the number of times we observe an emission of the symbol $b$ in state $k$. To go from counts to probabilities, we simply need relative frequences. Like, $e_k(b)=E_k(b)/(\sum_{b'}E_k(b'))$. 

But, the problem, of course, is that we just have observed sequence, and no path.

Consider now a problem where all that we have is an observed sequence. Ok, and the structure of the model. But no parameters, that is, transition and emission probabilities are not known. We would like to infer both from the emissions. This problem is called parameter estimation, and the process of inference of parameters from emission data the training of the model. Training of HMM is a special case of machine learing.

Here is an example:

In [1]:
T = {"F": {"F": 0.95, "L": 0.05},
     "L": {"F": 0.05, "L": 0.95}}
E = {"F": {"o": 0.5, "-": 0.5},
     "L": {"o": 0.7, "-": 0.3}}
start = "F"
states = E.keys()

import random
random.seed(42)

def weighted_choice(weighted_items):
    """Random choice given the list of elements and their weights"""
    rnd = random.random() * sum(weighted_items.values())
    for i, w in weighted_items.items():
        rnd -= w
        if rnd < 0:
            return i

def generate_hmm_sequence(h, T, E, n):
    """
    HMM sequence given start state,
    transition, emission matrix and sequence length
    """
    s = weighted_choice(E[h])
    yield h, s
    for _ in range(n-1):
        h = weighted_choice(T[h])
        yield h, weighted_choice(E[h])

In [2]:
path, x = list(zip(*generate_hmm_sequence(start, T, E, 50)))
print("\n".join(("".join(path), "".join(x))))

FFFFFFFFFFFFFFFFFLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
-o--ooo-o--o-o---o---ooo-oo-oo---ooo---o--o-oo-o--


In [3]:
from collections import Counter
c = Counter(generate_hmm_sequence(start, T, E, 10000))
c

Counter({('F', '-'): 2464,
         ('F', 'o'): 2389,
         ('L', 'o'): 3627,
         ('L', '-'): 1520})

In [4]:
def normalize(dic):
    s = sum(dic.values())
    return {k: dic[k]/s for k in dic}

e = {}
for state in states:
    e[state] = normalize({xi: c[pi, xi] for pi, xi in c if pi == state})
e

{'F': {'-': 0.5077271790644962, 'o': 0.4922728209355038},
 'L': {'o': 0.7046823392267341, '-': 0.295317660773266}}

## Viterbi Training

It is simple, but sort of very brute force. We initialize the procedure by setting $a_{kl}$ and $e_k(b)$ to some random values, taking care that these are probabilities and that some values have to some to 1. I know, it looks crazy, but we have to start somewhere.

At this stage, we have the model (the parameters) and the emitted sequence. We can use Viterbi decoding to compute the most probable path $\pi^*$. Now, we consider the newly computed $\pi^*$ and from it estimate the parameters of the model, $a_{kl}$ and $e_k(b)$. We now repeat the procedure, and do this until convergence, where $\pi^*$ is stable and does not change substantially.

Viterbi training looks simple but has one problem. Namely, we did not maximize the probability $P(x|\Theta)$, where we have denoted all the parameters of the model by $\Theta$. Instead, we maximized $P(x,\pi^*|\Theta)$, converging toward the parameters of the model where the probability of the most probable path is maximized.

One proper training procedure that maximizes $P(x|\Theta)$ is a [Baum-Welch algorithm](https://en.wikipedia.org/wiki/Baum–Welch_algorithm) and uses forward-backward algorithm that we have introduced above. At this stage, we are only informing you about this algorithm, and you are welcome to check it out and implement it yourself -- we here won't dive into details.